# Sandbox

### imports

In [1]:
from dotenv import load_dotenv
from ingestion.ingest import *

import sqlite3
import pandas as pd

from query_gen.functions import *

In [2]:
load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")

### ingestion

In [3]:
conn = sqlite3.connect('data/videos.db')

# View as DataFrame
df = pd.read_sql_query("SELECT * FROM videos", conn)
df.head()

,video_id,title,transcript
0,4oUOJ37GKYE,"The more they hurt you, the stronger you get #...",story is the story of the Hydra monster which ...
1,r5qk3uIdkks,What is #ai? — Simply Explained,what is AI exactly I work in the industry and ...
2,WgmMK5fS0X0,How to do MORE with LESS - multikills,multitasking isn't a good strategy how do we s...
3,5zeqo-R12vk,How to STAY dumb,in a lot of situations we might feel the need ...
4,hB27yAkJLC8,Being fragile means you have more downside tha...,first story is the sword of dimicles is the st...


In [4]:
df.shape

(158, 3)

In [5]:
video_id = 'vEvytl7wrGM'
irow = (df['video_id'] == video_id).idxmax()

### query gen

In [6]:
from ingestion.database import get_video_by_id
from query_gen.functions import get_all_comments, generate_queries

# Get video data
video_data = get_video_by_id(video_id)
print(f"Title: {video_data['title']}")

# Get comments
comments = get_all_comments(video_id)
print(f"Comments: {len(comments)}")

# Generate queries
result = generate_queries(
  video_title=video_data["title"],
  transcript=video_data["transcript"],
  comments=comments
)

# Display results
for i, q in enumerate(result.queries, 1):
  print(f"\n{i}. [{q.query_type.value} / {q.difficulty.value}]")
  print(f"   {q.query}")
  print(f"   → {q.grounding}")

Title: Claude Skills Explained in 23 Minutes
Comments: 6

1. [exact / grounded]
   What are Claude skills in Anthropic Claude?
   → Skill definition — "Claude skills? These are just reusable instructions"

2. [exact / medium]
   max length for skill desc?
   → Metadata and context — "metadata consists of the skills name and a short description"

3. [exact / hard]
   is there a limit to the number of sub agents u can make?
   → Sub agents feature — "specialized agents for specific workflows with their own context window"

4. [conceptual / grounded]
   Why do Claude skills help manage context window size?
   → Context management — "skills give us better context management without sacrificing capabilities"

5. [conceptual / medium]
   MCP or skills? which better for making Claude talk to other apps?
   → MCP vs Skills — "MCP allows any LLM to talk to any application while skills will only work with Claude"

6. [conceptual / hard]
   progressive discousre is basically token saving?
   → Pr

In [9]:
result.queries

[Query(query_type=<QueryType.EXACT: 'exact'>, difficulty=<DifficultyLevel.GROUNDED: 'grounded'>, grounding='Skill definition — "Claude skills? These are just reusable instructions"', query='What are Claude skills in Anthropic Claude?'),
 Query(query_type=<QueryType.EXACT: 'exact'>, difficulty=<DifficultyLevel.MEDIUM: 'medium'>, grounding='Metadata and context — "metadata consists of the skills name and a short description"', query='max length for skill desc?'),
 Query(query_type=<QueryType.EXACT: 'exact'>, difficulty=<DifficultyLevel.HARD: 'hard'>, grounding='Sub agents feature — "specialized agents for specific workflows with their own context window"', query='is there a limit to the number of sub agents u can make?'),
 Query(query_type=<QueryType.CONCEPTUAL: 'conceptual'>, difficulty=<DifficultyLevel.GROUNDED: 'grounded'>, grounding='Context management — "skills give us better context management without sacrificing capabilities"', query='Why do Claude skills help manage context windo